In [3]:
#install nltk
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [1]:
#import package
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [24]:
#load data
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
sample=pd.read_csv("sample_upload.csv")
train_X=train["tweet"]
train_Y=train["class"]
test_X=test["tweet"]

In [26]:
#add stopwords
stop_words=set(stopwords.words("english"))
stop_words.update(["RT","I"])

#text preprocessing
for i in range(len(train_X)):
    token=word_tokenize(train_X[i])
    for j in range(len(token)):
        if token[j]=="@" and j<len(token)-1:
            token[j+1]=":"
    train_X[i]=' '.join(token)
    
for i in range(len(test_X)):
    token=word_tokenize(test_X[i])
    for j in range(len(token)):
        if token[j]=="@" and j<len(token)-1:
            token[j+1]=":"
    test_X[i]=' '.join(token)

for i in range(len(train_X)):
    token=RegexpTokenizer(r"\w+").tokenize(train_X[i])
    token=[w for w in token if w not in stop_words]
    token=[w for w in token if w.isdigit()==False]
    train_X[i]=' '.join(token)
    
for i in range(len(test_X)):
    token=RegexpTokenizer(r"\w+").tokenize(test_X[i])
    token=[w for w in token if w not in stop_words]
    token=[w for w in token if w.isdigit()==False]
    test_X[i]=' '.join(token)

<ipython-input-26-1c6f335b5fc6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X[i]=' '.join(token)
<ipython-input-26-1c6f335b5fc6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_X[i]=' '.join(token)
<ipython-input-26-1c6f335b5fc6>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_X[i]=' '.join(token)
<ipython-input-26-1c6f335b5fc6>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [27]:
#TfidfVectorizer
tfidf=TfidfVectorizer()
tfidf.fit(train_X)
train_X=tfidf.transform(train_X)
test_X=tfidf.transform(test_X)

In [28]:
#training
clf=XGBClassifier(params = { 'max_depth': 6, 'objective': 'multi:softmax', 'num_class': 3 })
clf.fit(train_X,train_Y)
pred_Y=clf.predict(test_X)

C:\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:50:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "params" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:50:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [29]:
#create csv
outcome={"id":sample["id"],
         "Exited":pred_Y}
df_outcome=pd.DataFrame(outcome)
df_outcome
df_outcome.to_csv("predict_5.csv",index=False)